In [1]:
!wget https://ia803009.us.archive.org/10/items/201309_foursquare_dataset_umn/fsq.zip

--2020-04-05 07:05:23--  https://ia803009.us.archive.org/10/items/201309_foursquare_dataset_umn/fsq.zip
Resolving ia803009.us.archive.org (ia803009.us.archive.org)... 207.241.233.29
Connecting to ia803009.us.archive.org (ia803009.us.archive.org)|207.241.233.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160737760 (153M) [application/zip]
Saving to: ‘fsq.zip’

fsq.zip             100%[===================>] 153.29M   367KB/s    in 5m 14s  

2020-04-05 07:10:37 (500 KB/s) - ‘fsq.zip’ saved [160737760/160737760]



In [2]:
!unzip fsq.zip

Archive:  fsq.zip
   creating: umn_foursquare_datasets/
  inflating: umn_foursquare_datasets/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/umn_foursquare_datasets/
  inflating: __MACOSX/umn_foursquare_datasets/._.DS_Store  
  inflating: umn_foursquare_datasets/checkins.dat  
  inflating: umn_foursquare_datasets/ratings.dat  
  inflating: __MACOSX/umn_foursquare_datasets/._ratings.dat  
  inflating: umn_foursquare_datasets/socialgraph.dat  
  inflating: umn_foursquare_datasets/users.dat  
  inflating: umn_foursquare_datasets/venues.dat  


In [0]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('./umn_foursquare_datasets/checkins.dat', sep='|',skipinitialspace = True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data.columns = [name.strip() for name in data.columns]
data.dropna(inplace=True)
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


In [6]:
data.shape

(396634, 6)

In [0]:
from sklearn.cluster import MeanShift

In [0]:
data = data.iloc[0:100000]

In [0]:
model = MeanShift(bandwidth=0.1, n_jobs=-1)

coordinate_cols = ['latitude', 'longitude']
data = data[coordinate_cols]

In [10]:
%%time
data['cluster'] = model.fit_predict(data)

CPU times: user 36.9 s, sys: 7.1 s, total: 44 s
Wall time: 10min 4s


In [11]:
data

,latitude,longitude,cluster
2,38.895112,-77.036366,5
4,33.800745,-84.410520,7
8,45.523452,-122.676207,30
10,40.764462,-111.904565,65
11,33.448377,-112.074037,1
...,...,...,...
233789,33.575000,-117.725556,50
233790,37.629349,-122.400087,4
233794,29.762884,-95.383061,25
233798,32.802955,-96.769923,19


In [0]:
# delete clusters with less than 15 occurencies
n = 15
data = data[data.groupby('cluster')['cluster'].transform('count') >= n]

In [13]:
offices = np.array([
    [33.751277, -118.188740],
    [25.867736, -80.324116],
    [51.503016, -0.075479],
    [52.378894, 4.885084],
    [39.366487, 117.036146],
    [-33.868457, 151.205134]
])

centers = model.cluster_centers_
centers

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [ -37.8229826 ,  145.1811902 ],
       [ -41.2924945 ,  174.7732353 ],
       [ -45.0311622 ,  168.6626435 ]])

In [16]:
distances = np.zeros((offices.shape[0], centers.shape[0]))

for i, office in enumerate(offices):
    for j, center in enumerate(centers):
        distances[i, j] = np.linalg.norm(office - center)
distances

array([[ 44.74257092,   6.19395917,   6.29424146, ..., 272.92232369,
        302.42071128, 297.47334157],
       [ 16.14371999,  32.5726786 ,  32.47447468, ..., 234.32701671,
        263.78998313, 258.88426019],
       [ 74.69906582, 113.37331719, 113.2748328 , ..., 170.52458466,
        197.94716381, 194.40010686],
       [ 79.73425538, 118.41008135, 118.31160891, ..., 166.79141452,
        194.00080664, 190.55657439],
       [191.0327603 , 229.11470442, 229.01456477, ...,  82.16056067,
         99.19396543,  98.93562773],
       [237.22725876, 271.67953165, 271.58164907, ...,   7.20598166,
         24.70974973,  20.72126023]])

In [0]:
def smallestN_indices(a, N):
    idx = a.ravel().argsort()[:N]
    return np.stack(np.unravel_index(idx, a.shape)).T

In [50]:
k = 20

ixs = smallestN_indices(distances, k)

distances[ixs[:,0], ixs[:,1]]

array([0.00783476, 0.00935332, 0.02267407, 0.05005829, 0.07084773,
       0.13410903, 0.15410283, 0.16740596, 0.18887596, 0.19222727,
       0.19577946, 0.21181054, 0.22223329, 0.24065315, 0.25150209,
       0.26892863, 0.27130076, 0.27531986, 0.28971896, 0.29497889])

In [49]:
centers[420]

array([-33.86063043, 151.20477593])